In [1]:
import psycopg2

conn = psycopg2.connect(
    "dbname=interviewhell",
    user="postgres",
    password="zaq12WSX",
    host="localhost",
    port="5432"
    )

cursor = conn.cursor()
cursor.execute("select joboffers.title, categories.name, questions.text from questions, joboffers, categories;")

result = cursor.fetchall()
print(result)
conn.close()

[("Satan's Butt Scratcher", 'HR', "Why do you think you are the best fit for scratching Satan's butt?"), ("Satan's Butt Scratcher", 'Technical', "Why do you think you are the best fit for scratching Satan's butt?"), ("Satan's Butt Scratcher", 'Problem Solving', "Why do you think you are the best fit for scratching Satan's butt?"), ("Satan's Butt Scratcher", 'HR', 'What motivates you to work in such a personal role?'), ("Satan's Butt Scratcher", 'Technical', 'What motivates you to work in such a personal role?'), ("Satan's Butt Scratcher", 'Problem Solving', 'What motivates you to work in such a personal role?'), ("Satan's Butt Scratcher", 'HR', 'How do you handle situations where Satan might be unhappy with your work?'), ("Satan's Butt Scratcher", 'Technical', 'How do you handle situations where Satan might be unhappy with your work?'), ("Satan's Butt Scratcher", 'Problem Solving', 'How do you handle situations where Satan might be unhappy with your work?'), ("Satan's Butt Scratcher", 

In [2]:
train_data = []
formatted_data_boilerplate = """
Job Title: {}
Question Category: {}
Interview Question: {}
"""
for row in result:
    train_data.append(formatted_data_boilerplate.format(row[0], row[1], row[2]))

print(train_data)

["\nJob Title: Satan's Butt Scratcher\nQuestion Category: HR\nInterview Question: Why do you think you are the best fit for scratching Satan's butt?\n", "\nJob Title: Satan's Butt Scratcher\nQuestion Category: Technical\nInterview Question: Why do you think you are the best fit for scratching Satan's butt?\n", "\nJob Title: Satan's Butt Scratcher\nQuestion Category: Problem Solving\nInterview Question: Why do you think you are the best fit for scratching Satan's butt?\n", "\nJob Title: Satan's Butt Scratcher\nQuestion Category: HR\nInterview Question: What motivates you to work in such a personal role?\n", "\nJob Title: Satan's Butt Scratcher\nQuestion Category: Technical\nInterview Question: What motivates you to work in such a personal role?\n", "\nJob Title: Satan's Butt Scratcher\nQuestion Category: Problem Solving\nInterview Question: What motivates you to work in such a personal role?\n", "\nJob Title: Satan's Butt Scratcher\nQuestion Category: HR\nInterview Question: How do you 

In [3]:
from transformers import GPT2Tokenizer
from datasets import Dataset

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the list of prompts
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=200)

# Convert list to Hugging Face dataset format
dataset = Dataset.from_dict({"text": train_data}).map(tokenize_function, batched=True)

/Users/wojciechbos/Documents/GitHub/Interview-hell-ai/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Map: 100%|██████████| 2898/2898 [00:00<00:00, 5452.90 examples/s]


In [4]:
from transformers import GPT2LMHeadModel

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Assign the padding token to avoid errors
model.config.pad_token_id = tokenizer.pad_token_id


In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./gpt2-job-interviews",
    overwrite_output_dir=True,
    num_train_epochs=3,        # You can increase this if needed
    per_device_train_batch_size=2,  # Adjust batch size based on your GPU/CPU
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50
)


In [6]:
from transformers import Trainer, DataCollatorForLanguageModeling

# Create a data collator (ensures padding is handled correctly)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # We are training GPT-2, not using masked language modeling (MLM)
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)


In [7]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,2.206900
100,1.064400
150,0.597400
200,0.493500
250,0.421100
300,0.381000
350,0.346200
400,0.343900
450,0.333300
500,0.322000


TrainOutput(global_step=4347, training_loss=0.30593907983927776, metrics={'train_runtime': 3482.2461, 'train_samples_per_second': 2.497, 'train_steps_per_second': 1.248, 'total_flos': 887372236800000.0, 'train_loss': 0.30593907983927776, 'epoch': 3.0})

In [8]:
model.save_pretrained("./gpt2-job-interview-finetuned")
tokenizer.save_pretrained("./gpt2-job-interview-finetuned")


('./gpt2-job-interview-finetuned/tokenizer_config.json',
 './gpt2-job-interview-finetuned/special_tokens_map.json',
 './gpt2-job-interview-finetuned/vocab.json',
 './gpt2-job-interview-finetuned/merges.txt',
 './gpt2-job-interview-finetuned/added_tokens.json')

In [9]:
from transformers import pipeline

# Load the fine-tuned model
generator = pipeline("text-generation", model="./gpt2-job-interview-finetuned", tokenizer=tokenizer)

# Generate an interview question for a new job title
prompt = """
Job Title: Hell Plumber
Question Category: HR
Interview Question:
"""
generated_text = generator(prompt, max_length=100, num_return_sequences=1)

print(generated_text[0]["generated_text"])


Device set to use mps:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



Job Title: Hell Plumber
Question Category: HR
Interview Question:
Job Title: Eternal Screams DJ
Interview Question: How would you improvise if a new regulation banned the use of traditional fire techniques?
Interview Question: How would you handle a situation where lava spilled during your task?
Interview Question: How do you innovate in the field of demonic torment targeting specific traits?
Interview Question: How do you innovate in the field of demonic torment targeting specific traits?
Interview


In [11]:
prompt = """
Job Title: Torture Project Manager
Question Category: HR
Interview Question:
"""
generated_text = generator(
    prompt, 
    max_length=50,       # Reduce length to prevent excessive output
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,  # Ensures GPT-2 stops at the end of a question
    temperature=0.7,     # Controls randomness (lower = more predictable)
    top_p=0.9,           # Nucleus sampling (helps avoid too random outputs)
    top_k=50,            # Limits vocabulary choices for better structure
    do_sample=True       # Ensures diverse results
)

print(generated_text[0]["generated_text"])



Job Title: Torture Project Manager
Question Category: HR
Interview Question:
Job Title: Ash Sorting Specialist
Interview Question: Describe your approach to managing lava currents.
Interview Question: How do you handle criticism from demons about
